Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = "Maciej WIlhelmi"

---

# 4. Metody tradycyjne - zadania do rozwiązania

In [37]:
%run "2. Zadania w przetwarzaniu grafów.ipynb"

{'train_auc': 0.7447835710684279, 'test_auc': 0.5119679007496767}
{'train_auc': 0.5722406442444339, 'test_auc': 0.5465309022022259}
{'train_auc': 0.9529060072208523, 'test_auc': 0.5259835748704671}


### Zadanie 4.1. (2.5 pkt)
W poprzednim zeszycie zbadaliśmy jakość reprezentacji otrzymywanych za pomocą metody DeepWalk. W tym zadaniu zbadaj wpływ parametrów `p` i `q` na jakość działania metody Node2vec ($p \neq 1$ oraz $q \neq 1$):
- wybierz kilka (min. 4) wartości dla każdego parametru
- podsumuj w tabelce jakość wyznaczonych reprezentacji w zadaniu klasyfikacji wierzchołków na zbiorze Cora
- pamiętaj, aby eksperyment dla każdego pojedynczego zestawu parametrów powtórzyć kilkukrotnie (np. 5 razy)
- wyniki podaj w postaci: "średnie testowe AUC $\pm$ odchylenie standardowe".

Który zestaw parametrów zadziałał najlepiej?

In [2]:
import pandas as pd
from random_walk_model import get_representations
from torch_geometric.datasets import Planetoid
from random_walk_model import train_random_walk_model
import numpy as np
from prettytable import PrettyTable

In [ ]:

p_arr = [1]
q_arr = [1]

for p in p_arr:
    for q in q_arr:
        deepwalk, train_losses = train_random_walk_model(edge_index=data.edge_index, p=p, q=q)
        z = get_representations(deepwalk)
        nc_metrics = evaluate_node_classification(data=data, z=z)
        
        train_auc.append(nc_metrics['train_auc'])
        test_auc.append(nc_metrics['test_auc'])
        
        
        print(f"-- Node classification -- for p={p} and q={q}")
        print(f"Train AUC: {nc_metrics['train_auc']} [%]")
        print(f"Test AUC: {nc_metrics['test_auc'] * 100.:.2f} [%]")

In [6]:
nc_metrics = evaluate_node_classification(data=data, z=z)

print(f"-- Node classification -- for p=1 and q=1")
print(f"Train AUC: {nc_metrics['train_auc'] * 100.:.2f} [%]")
print(f"Test AUC: {nc_metrics['test_auc'] * 100.:.2f} [%]")

-- Node classification -- for p=1 and q=1
Train AUC: 97.98 [%]
Test AUC: 93.72 [%]


In [6]:
# TU WPISZ KOD

data = Planetoid(root="./data/", name="Cora")[0]

In [12]:
p = [0.5, 2]
q = [1.5, 0.5]

means = {}

for p_v in p:
  for q_v in q:
    mean_train_auc = []
    mean_test_auc = []
    for i in range(2):
      lp_deepwalk, _ = train_random_walk_model(edge_index=data.edge_index, p=p_v, q=q_v, num_epochs=10)

      z = get_representations(lp_deepwalk)
      nc_metrics = evaluate_node_classification(data=data, z=z)
    
      mean_train_auc.append(np.round(nc_metrics['train_auc'] * 100, 3))
      mean_test_auc.append(np.round(nc_metrics['test_auc'] * 100, 3))
    means[p_v,q_v] = [np.mean(mean_train_auc), np.mean(mean_test_auc)]


tabel = PrettyTable()
tabel.field_names = ['[p,q]', 'Train AUC [%]', 'Test AUC [%]']

for key, value in means.items():
    tabel.add_row([key, value[0], value[1]])

print(tabel)

+------------+-------------------+-------------------+
|   [p,q]    |   Train AUC [%]   |    Test AUC [%]   |
+------------+-------------------+-------------------+
| (0.5, 1.5) | 80.10249999999999 |      62.6315      |
| (0.5, 0.5) |       81.593      |       63.449      |
|  (2, 1.5)  |      81.9785      | 64.00450000000001 |
|  (2, 0.5)  |      81.8545      |      64.3125      |
+------------+-------------------+-------------------+


Najlepiej z przebadanych zestawów działa ten, w którym p=2 > q=0.5, jednak nie zbadałem dużej ich ilości ze względu na długość uczenia, również zmniejszyłem ilość epok ze 100 na 10 aby otrzymać wyniki i uśredniłem je po dwóch przejściach dla tych samych parametrów.

### Zadanie 4.2. (2 pkt)
Korzystając z wyznaczonych najlepszych parametrów dla Node2vec'a, porównaj jakość reprezentacji krawędzi otrzymywanych za pomocą różnych transformacji:
- wykorzystaj zbiór Cora
- zbadaj 4 transformacje: "average", "hadamard", "L1" oraz "L2"
- każdy eksperyment powtórz kilkukrotnie (np. 5 razy)
- podsumuj wyniki w tabelce, zapisując wynik pojedynczego eksperymentu w postaci: "średnie testowe AUC $\pm$ odchylenie standardowe"

Która transformacja działa najlepiej?

In [14]:
# TU WPISZ KOD
lp_dataset = prepare_train_test_sets(edge_index=data.edge_index)

transformations = ['average', 'hadamard', 'L1', 'L2']

means = {}

for transform in transformations:
    for i in range(2):
      lp_deepwalk, _ = train_random_walk_model(edge_index=data.edge_index, p=2, q=0.5, num_epochs=10)

      lp_z = get_representations(lp_deepwalk)
        
      lp_metrics = evaluate_link_prediction(
          train_edges=lp_dataset["train_edges"],
            y_train=lp_dataset["y_train"],
            test_edges=lp_dataset["test_edges"],
            y_test=lp_dataset["y_test"],
            transformation_name=transform,
            z=lp_z,
      )

      mean_train_auc.append(np.round(lp_metrics['train_auc'] * 100, 3))
      mean_test_auc.append(np.round(lp_metrics['test_auc'] * 100, 3))
    means[transform] = [np.mean(mean_train_auc), np.mean(mean_test_auc)]


tabel = PrettyTable()
tabel.field_names = ['transformation', 'Train AUC [%]', 'Test AUC [%]']

for key, value in means.items():
    tabel.add_row([key, value[0], value[1]])

print(tabel)

Liczba parametrów modelu: 346,624


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Liczba parametrów modelu: 346,624


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Liczba parametrów modelu: 346,624


...

Liczba parametrów modelu: 346,624


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Liczba parametrów modelu: 346,624


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

+----------------+-------------------+--------------------+
| transformation |   Train AUC [%]   |    Test AUC [%]    |
+----------------+-------------------+--------------------+
|    average     |      70.40575     | 60.764500000000005 |
|    hadamard    | 73.05166666666666 | 66.27533333333334  |
|       L1       | 70.89387500000001 |      65.4965       |
|       L2       | 69.51240000000001 |      64.8409       |
+----------------+-------------------+--------------------+


Dla parametrów z poprzedniego zadania sprawdziłem sposoby przekształcania reprezentacji wierzchołków na krawędzie. Tutaj dosyć wyraźnie wygrał 'hadamard', który mnoży reprezentacje wektorów, tworząc reprezentacje krawędzi.

### Zadanie 4.3. (1.5 pkt)
Korzystając z wyznaczonych najlepszych parametrów dla Node2vec'a, porównaj jakość reprezentacji grafów otrzymywanych za pomocą różnych transformacji:
- wykorzystaj zbiór ENZYMES (może być podzbiór wykorzystany w poprzednim zeszycie)
- zbadaj 3 transformacje: "average", "max_pooling", "min_pooling"
- każdy eksperyment powtórz kilkukrotnie (np. 5 razy)
- podsumuj wyniki w tabelce, zapisując wynik pojedynczego eksperymentu w postaci: "średnie testowe AUC $\pm$ odchylenie standardowe"

Która transformacja działa najlepiej?

In [5]:
from torch_geometric.datasets import TUDataset
from tqdm.auto import tqdm

enzymes = TUDataset(root="./data", name="ENZYMES")

print(f"Liczba grafów: {len(enzymes)}")

enzymes_subset, _ = train_test_split(
    [e for e in enzymes], 
    train_size=0.1, 
    stratify=[e.y.item() for e in enzymes],
)
print(f"Ograniczymy liczbę grafów do {len(enzymes_subset)}")

Liczba grafów: 600
Ograniczymy liczbę grafów do 60


In [9]:
# TU WPISZ KOD

z = []

for enzyme in tqdm(enzymes_subset, desc="Embedding graphs"):
    model, _ = train_random_walk_model(
        edge_index=enzyme.edge_index,
        p=2,
        q=0.5,
        num_epochs=5,
        quiet=True,
    )
    
    z.append(get_representations(model))
        
        
y = torch.tensor([e.y for e in enzymes_subset])

Embedding graphs:   0%|          | 0/60 [00:00<?, ?it/s]

In [16]:
len(y)

60

In [38]:
transformations = ["average", "max_pooling", "min_pooling"]

means = {}
mean_train_auc = []
mean_test_auc = []

for transform in transformations:
    for i in range(10):
        gc_metrics = evaluate_graph_classification(z, y, transformation_name=transform)

        mean_train_auc.append(gc_metrics['train_auc'] * 100)
        mean_test_auc.append(gc_metrics['test_auc'] * 100)
        
    means[transform] = [np.round(np.mean(mean_train_auc),3)
                        , np.round(np.mean(mean_test_auc),3)]


tabel = PrettyTable()
tabel.field_names = ['transformation', 'Train AUC [%]', 'Test AUC [%]']

for key, value in means.items():
    tabel.add_row([key, value[0], value[1]])

print(tabel)

+----------------+---------------+--------------+
| transformation | Train AUC [%] | Test AUC [%] |
+----------------+---------------+--------------+
|    average     |     100.0     |    49.164    |
|  max_pooling   |     100.0     |    48.117    |
|  min_pooling   |     100.0     |    46.713    |
+----------------+---------------+--------------+


najlepiej zadziałała transformacja 'average'.